In [2]:
class Node:
    def __init__(self, node_type, left=None, right=None, value=None):
        self.type = node_type  # "operator" or "operand"
        self.left = left  # Left child node
        self.right = right  # Right child node
        self.value = value  # Value for operand nodes (None for operator nodes)

    def __repr__(self):
        return f"Node({self.type}, {self.left}, {self.right}, {self.value})"

# Example of how to create an AST manually
# Rule: (age > 30 AND department = 'Sales')
node = Node("AND",
            left=Node("operand", value="age > 30"),
            right=Node("operand", value="department = 'Sales'"))
print(node)


Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None)


In [3]:
import re

def create_rule(rule_string):
    # Parse the rule using regex and build the AST
    if "AND" in rule_string:
        operator = "AND"
    elif "OR" in rule_string:
        operator = "OR"
    else:
        raise ValueError("Unsupported operator")

    operands = re.split(f"\s{operator}\s", rule_string)
    left_operand = operands[0].strip()
    right_operand = operands[1].strip()
    
    return Node(operator, 
                left=Node("operand", value=left_operand), 
                right=Node("operand", value=right_operand))

rule1 = "age > 30 AND department = 'Sales'"
ast = create_rule(rule1)
print(ast)


Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None)


In [4]:
def combine_rules(rule_strings):
    combined_rule = None
    for rule in rule_strings:
        ast = create_rule(rule)
        if combined_rule:
            combined_rule = Node("AND", left=combined_rule, right=ast)
        else:
            combined_rule = ast
    return combined_rule

rules = ["age > 30 AND department = 'Sales'", "salary > 50000 OR experience > 5"]
combined_ast = combine_rules(rules)
print(combined_ast)


Node(AND, Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None), Node(OR, Node(operand, None, None, salary > 50000), Node(operand, None, None, experience > 5), None), None)


In [5]:
def evaluate_node(node, data):
    if node.type == "operand":
        # Parse and evaluate the operand
        key, operator, value = re.split(r"\s(>|<|=)\s", node.value)
        key = key.strip()
        value = value.strip().replace("'", "")
        if operator == ">":
            return data[key] > int(value)
        elif operator == "<":
            return data[key] < int(value)
        elif operator == "=":
            return data[key] == value
    elif node.type == "AND":
        return evaluate_node(node.left, data) and evaluate_node(node.right, data)
    elif node.type == "OR":
        return evaluate_node(node.left, data) or evaluate_node(node.right, data)
    return False

# Test evaluation
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
print(evaluate_node(combined_ast, data))


True


In [6]:
# Test Case 1: Single rule creation
rule1_ast = create_rule("age > 30 AND department = 'Sales'")
print(rule1_ast)

# Test Case 2: Combine rules
combined_ast = combine_rules(["age > 30 AND department = 'Sales'", "salary > 50000 OR experience > 5"])
print(combined_ast)

# Test Case 3: Evaluate rule with JSON data
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
print(evaluate_node(combined_ast, data))  # Expected: True


Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None)
Node(AND, Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None), Node(OR, Node(operand, None, None, salary > 50000), Node(operand, None, None, experience > 5), None), None)
True


In [8]:
#BONUS
import re

class InvalidRuleException(Exception):
    pass

def create_rule(rule_string):
    # Check if the rule contains a valid operator (AND/OR)
    if "AND" not in rule_string and "OR" not in rule_string:
        raise InvalidRuleException("Missing operator in the rule string. Rule must contain AND or OR.")
    
    # Parse the rule using regex and build the AST
    operator = "AND" if "AND" in rule_string else "OR"
    
    # Split the operands
    operands = re.split(f"\s{operator}\s", rule_string)
    
    # Ensure there are exactly two operands
    if len(operands) != 2:
        raise InvalidRuleException(f"Invalid rule format: {rule_string}")
    
    # Validate operand format (simple validation, you can expand this for more complex cases)
    def validate_operand(operand):
        if not re.match(r"^[a-zA-Z_]+\s*(>|<|=)\s*[0-9a-zA-Z'_]+$", operand):
            raise InvalidRuleException(f"Invalid comparison in operand: {operand}")
    
    left_operand = operands[0].strip()
    right_operand = operands[1].strip()
    
    validate_operand(left_operand)
    validate_operand(right_operand)
    
    return Node(operator, 
                left=Node("operand", value=left_operand), 
                right=Node("operand", value=right_operand))

# Test with a valid rule
try:
    rule1 = "age > 30 AND department = 'Sales'"
    ast = create_rule(rule1)
    print(ast)
except InvalidRuleException as e:
    print(e)

# Test with an invalid rule (missing operator)
try:
    invalid_rule = "age > 30 department = 'Sales'"
    ast = create_rule(invalid_rule)
except InvalidRuleException as e:
    print(f"Error: {e}")


Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None)
Error: Missing operator in the rule string. Rule must contain AND or OR.


In [9]:
valid_attributes = ["age", "department", "salary", "experience"]

def validate_operand(operand):
    # Extract the attribute (everything before the operator)
    match = re.match(r"^([a-zA-Z_]+)\s*(>|<|=)\s*[0-9a-zA-Z'_]+$", operand)
    if not match:
        raise InvalidRuleException(f"Invalid comparison in operand: {operand}")
    
    attribute = match.group(1)
    if attribute not in valid_attributes:
        raise InvalidRuleException(f"Invalid attribute '{attribute}' used in operand. Must be one of {valid_attributes}.")

# Test with an invalid attribute
try:
    invalid_rule = "height > 180 AND department = 'Sales'"
    ast = create_rule(invalid_rule)
except InvalidRuleException as e:
    print(f"Error: {e}")


In [10]:
def modify_rule(ast, modification_type, new_value):
    """
    Modify an existing rule's AST.
    
    :param ast: The root of the AST to modify.
    :param modification_type: Type of modification ("change_operator", "change_operand", "add_subexpression").
    :param new_value: New value or node to modify the AST with.
    :return: Modified AST.
    """
    if modification_type == "change_operator":
        if ast.type == "AND" or ast.type == "OR":
            ast.type = new_value  # Change the operator ("AND" or "OR")
        else:
            raise InvalidRuleException("Cannot change operator on non-operator node.")
    elif modification_type == "change_operand":
        if ast.type == "operand":
            ast.value = new_value  # Change the operand value
        else:
            raise InvalidRuleException("Cannot change operand on non-operand node.")
    elif modification_type == "add_subexpression":
        if ast.type == "AND" or ast.type == "OR":
            new_node = create_rule(new_value)  # Create a new subexpression as an AST
            ast.right = Node(ast.type, left=ast.right, right=new_node)  # Add new subexpression as right child
        else:
            raise InvalidRuleException("Can only add sub-expressions to operator nodes.")
    else:
        raise InvalidRuleException("Invalid modification type.")
    
    return ast

# Test rule modification (Change Operator)
try:
    rule_ast = create_rule("age > 30 AND department = 'Sales'")
    print("Original AST:", rule_ast)
    modified_ast = modify_rule(rule_ast, "change_operator", "OR")
    print("Modified AST (Operator Changed):", modified_ast)
except InvalidRuleException as e:
    print(f"Error: {e}")


Original AST: Node(AND, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None)
Modified AST (Operator Changed): Node(OR, Node(operand, None, None, age > 30), Node(operand, None, None, department = 'Sales'), None)


In [11]:
# Define a dictionary of user-defined functions
user_defined_functions = {
    "is_high_income": lambda salary: salary > 100000,
    "is_senior_employee": lambda experience: experience > 10
}

def evaluate_node(node, data, user_functions=None):
    if node.type == "operand":
        # Parse and evaluate the operand
        key, operator, value = re.split(r"\s(>|<|=)\s", node.value)
        key = key.strip()
        value = value.strip().replace("'", "")
        
        if key in user_functions:
            # If key is a user-defined function, use it
            return user_functions[key](data.get(key))
        
        if operator == ">":
            return data[key] > int(value)
        elif operator == "<":
            return data[key] < int(value)
        elif operator == "=":
            return data[key] == value
    elif node.type == "AND":
        return evaluate_node(node.left, data, user_functions) and evaluate_node(node.right, data, user_functions)
    elif node.type == "OR":
        return evaluate_node(node.left, data, user_functions) or evaluate_node(node.right, data, user_functions)
    return False

# Test evaluation with user-defined function
data = {"salary": 120000, "experience": 15}
try:
    rule_with_func = "is_high_income AND is_senior_employee"
    ast_with_func = create_rule(rule_with_func)
    result = evaluate_node(ast_with_func, data, user_defined_functions)
    print(f"Evaluation result with user-defined functions: {result}")
except InvalidRuleException as e:
    print(f"Error: {e}")


Error: Invalid comparison in operand: is_high_income
